Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [ ]:
import os, re
from getpass import getpass

with open('nils-project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

# password read from file

Install required packages and dependencies. Ignore the warnings.

In [ ]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

Import packages and define custom functions

In [ ]:
import json
from loguru import logger
import chi.network, chi.container
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, get_available_publicips, get_worker_interfaces, get_segment_ids

def get_reservation_id_by_name(name):
    for lease in list_reservations(brief=True):
        if name in lease['name']:
            return lease['reservation_id']

def get_available_interface(worker_name, number=1):
    interfaces = list(get_worker_interfaces(worker_name).values())[0]
    available_ifs = []
    for interface in interfaces.keys():
        if len(interfaces[interface]['connections']) == 0:
            available_ifs.append(interface)
    if len(available_ifs) < number:
        logger.info(f"{json.dumps(interfaces, indent=4)}")
        raise Exception(f"Did not find enough interfaces on {worker_name}")
    return sorted(available_ifs)[:number]

def get_network_id_by_name(name):
    for network in chi.network.list_networks():
        if name in network['name']:
            return network['id']
    raise Exception(f"Network {name} not found")

def get_segment_id(name):
    result = get_segment_ids(name)
    return next(iter(result.values()))

Project settings

In [ ]:
# Fixed IP addresses
ep5g_gw_addr = "10.30.111.10"
public_gw_addr = "130.237.11.97"
ue_gw_addr = "10.42.3.1"
ue_priv_addr = "10.42.3.2"
ue_cidr = "172.16.0.0/16"
adv_nat_addr = {
    "adv-01": "172.16.0.8",
    "adv-02": "172.16.0.88",
    "adv-03": "172.16.0.40",
    "adv-04": "172.16.0.96",
    "adv-05": "172.16.0.64",
    "adv-06": "172.16.0.72",
    "adv-07": "172.16.0.104",
    "adv-08": "172.16.0.56",
}

# User-defined edge network
edge_cidr = "10.70.70.0/24"
edge_gw_addr = "10.70.70.1"
edge_h1_addr = "10.70.70.10"
edge_h2_addr = "10.70.70.20"
edge_h3_addr = "10.70.70.30"
edge_h4_addr = "10.70.70.40"

# Routes
route_from_edge_to_ue = "-".join([ue_cidr,edge_gw_addr])
route_from_ue_to_edge = "-".join([edge_cidr,ue_gw_addr])

# Assigning workers
edge_node_worker ="worker-01"
ue1_node_worker = "worker-08"

# Assigning router
ue1_adv_name = "adv-04"
ue1_nat_addr = adv_nat_addr[ue1_adv_name]

Reserve the required equipment and resources

In [ ]:
experiment_duration = {"days": 0, "hours": 16}

# List of required leases with details
required_leases = [
    { 
        "type": "network",
        "name": "ep5g",
        "net_name": "ep5g-vip",
        "segment_id": get_segment_id("ep5g"),
        "duration": experiment_duration
    },
    {
        "type": "network",
        "name": ue1_adv_name,
        "net_name": ue1_adv_name,
        "segment_id": get_segment_id(ue1_adv_name),
        "duration": experiment_duration,
    },
    {
        "type": "device",
        "name": edge_node_worker,
        "duration": experiment_duration,
    },
    {
        "type": "device",
        "name": ue1_node_worker,
        "duration": experiment_duration,
    },
]

# List of previously existing leases
existing_leases = list_reservations(brief=True)

# Reserve outstanding resources
for required_lease in required_leases:
    lease_name_with_suffix = required_lease["name"] + "-lease"

    # Check if the resource is already leased
    is_already_leased = False
    for existing_lease in existing_leases:
        if existing_lease["name"] == lease_name_with_suffix:
            is_already_leased = True
            break

    # If it is already leased, check the lease status
    if is_already_leased and existing_lease["status"] == "ACTIVE":
        logger.info(f"Resource {required_lease['name']} is already leased and ACTIVE.")
        continue
    if is_already_leased and existing_lease["status"] == "TERMINATED":
        logger.info(f"Removing TERMINATED lease of {existing_lease['name']} before proceeding.")
        unreserve_byid(existing_lease["id"])
    
    # If it is NOT already leased, reserve it (default case)
    reserve(required_lease)

Create networks and routers

In [ ]:
# Create networks
try: 
    edgenet = chi.network.get_network("edge-net")
    edge_subnet = chi.network.get_subnet("edge-subnet")
    logger.info("edge-net already exists.")
except:
    edgenet = chi.network.create_network("edge-net")
    edge_subnet = chi.network.create_subnet(
        subnet_name = "edge-subnet", 
        network_id = edgenet["id"], 
        cidr = edge_cidr, 
        gateway_ip = edge_gw_addr, 
        enable_dhcp = False
    )
    logger.success("edge-net is created.")

# Create router from edge-net to epg5
ep5g_net = chi.network.get_network("ep5g-vip-net")

try:
    chi.network.get_router("edge-router")
    logger.info("Router already exists.")
except:
    router = chi.network.create_router("edge-router", "public")
    chi.network.add_subnet_to_router(router["id"], edge_subnet["id"])
    chi.network.add_subnet_to_router(router["id"], ep5g_net["subnets"][0])
    chi.network.add_route_to_router(router["id"], ue_cidr, ep5g_gw_addr)
    logger.success("Router created and subnets added.")

Start server on edge

In [ ]:
# edge_node_addr_public = get_available_publicips()[-1]
# logger.info(f"Public IP address will be {edge_node_addr_public} for this container.")

edge_node_interfaces = get_available_interface(edge_node_worker, number=2)
logger.info(f"Using interface(s): {edge_node_interfaces}")

edge_node_container_name = "edge-server"
edge_node_image_name = "nilsjor/ros-humble-turtlebot:husarnet-edge-server-v1.1"

with open("husarnet-joincode.txt", 'r') as file:
    edge_node_env_vars = {
        "HUSARNET_JOIN_CODE": file.read().strip(),
        "HUSARNET_HOSTNAME": edge_node_container_name,
        "DNS_IP": "1.1.1.1",
        "GATEWAY_IP": edge_gw_addr,
        # "GATEWAY_IP": public_gw_addr,
    }

edge_node_labels = {
    "networks.1.interface": edge_node_interfaces[0],
    "networks.1.routes": route_from_edge_to_ue,
    "networks.1.ip": edge_h1_addr + "/24",
    "networks.1.gateway": edge_gw_addr,
    # "networks.2.interface": edge_node_interfaces[1],
    # "networks.2.ip": edge_node_addr_public + "/27",
    # "networks.2.gateway": public_gw_addr,
    "capabilities.privileged": "true",
}

try:
    chi.container.destroy_container(edge_node_container_name)
    wait_until_container_removed(edge_node_container_name)
    logger.success("Previous container destroyed.")
except:
    logger.info("No previous container found.")

edge_node_container = chi.container.create_container(
    name = edge_node_container_name,
    image = edge_node_image_name,
    reservation_id = get_reservation_id_by_name(edge_node_worker),
    environment = edge_node_env_vars,
    nets = [
        { "network": edgenet['id'] },
        # { "network": get_network_id_by_name("serverpublic") },
    ],
    labels = edge_node_labels,
)
chi.container.wait_for_active(edge_node_container_name)
logger.success("Container deployed and active.")

Start UE1

In [ ]:
ue_node_interfaces = get_available_interface(ue1_node_worker)
logger.info(f"Using interface(s): {ue_node_interfaces}")

ue_node_container_name = "ue-node"
ue_node_image_name = "nilsjor/ros-humble-turtlebot:husarnet-device-node-v1.1"

with open("husarnet-joincode.txt", 'r') as file:
    ue_node_env_vars = {
        "HUSARNET_JOIN_CODE": file.read().strip(),
        "HUSARNET_HOSTNAME": ue_node_container_name,
        "DNS_IP": "1.1.1.1",
        "GATEWAY_IP": ue_gw_addr,
    }

ue_node_labels = {
    "networks.1.interface": ue_node_interfaces[0],
    "networks.1.ip": ue_priv_addr + "/24",
    "networks.1.routes": route_from_ue_to_edge,
    "networks.1.gateway": ue_gw_addr,
    "capabilities.privileged": "true",
}

try:
    chi.container.destroy_container(ue_node_container_name)
    wait_until_container_removed(ue_node_container_name)
    logger.success("Previous container destroyed.")
except:
    logger.info("No previous container found.")

ue_node_container = chi.container.create_container(
    name = ue_node_container_name,
    image = ue_node_image_name,
    reservation_id = get_reservation_id_by_name(ue1_node_worker),
    environment = ue_node_env_vars,
    nets = [
        { "network": get_network_id_by_name(ue1_adv_name) },
    ],
    labels = ue_node_labels,
)

chi.container.wait_for_active(ue_node_container_name)
logger.success("Container deployed and active.")

TODO: Reconfigure router to disconnect UE1 from the public internet

## Teardown

Destroy all containers

In [ ]:
try:
    status = get_container_status(edge_node_container_name)
    chi.container.destroy_container(edge_node_container_name)
    wait_until_container_removed(edge_node_container_name)
except:
    logger.info("No edge-node container found.")

try:
    status = get_container_status(ue_node_container_name)
    chi.container.destroy_container(ue_node_container_name)
    wait_until_container_removed(ue_node_container_name)
except:
    logger.info("No ue-node container found.")

logger.info("Stopped and removed all containers")

Manually check if any dangling ports are left behind. Enter their ID (not name) in the next cell and run it to delete them.

In [ ]:
dangling_ports = [
    
]

for port in dangling_ports:
    try:
        chi.network.delete_port(port)
    except:
        logger.info(f"Port {port} not found.")

Proceed to clean up the rest of the project.

In [ ]:
# find the router again
router = None
try:
    router = chi.network.get_router("edge-router")
except Exception as ex:
    logger.info("Could not find edge-router.")

if router:
    # remove all routes from the router
    chi.network.remove_all_routes_from_router(router["id"])
    logger.success("Removed all routes from router.")

    # remove all subnets from the router
    subnets = chi.network.list_subnets()
    logger.info(f"Checking all {len(subnets)} subnets.")
    for subnet in subnets:
        try:
            chi.network.remove_subnet_from_router(router["id"],subnet["id"])
        except Exception as ex:
            pass
    logger.success("Removed all subnets from router")

    chi.network.delete_router(router["id"])
    logger.success("Deleted the router")

edgenet = None
try:
    edgenet = chi.network.get_network("edge-net")
except Exception as ex:
    logger.info("Could not find edge-net.")

if edgenet:
    chi.network.delete_network(edgenet["id"])
    logger.success("Deleted the edge-net")

Terminate reservations

In [ ]:
leaseslist = list_reservations(brief=True)
for lease in leaseslist:
    unreserve_byid(lease["id"])
    logger.success("Removed " + lease["name"])

logger.info("no leases remaining.")